In [67]:
import pandas as pd

times = 1.5
fcst_path = "rolling_predict_non_quantile.csv"
fcst_q_98 = "rolling_predict_quantile.csv"
fcst_q_9 = 'rolling_predict_quantile_0.9.csv'
fcst_q_92 = 'rolling_predict_quantile_0.92.csv'
fcst_q_94 = 'rolling_predict_quantile_0.94.csv'
fcst_q_96 = 'rolling_predict_quantile_0.96.csv'
save_path = "result_0.98_20190701.csv"

fcst = pd.read_csv(fcst_path)
fcst_q_98 = pd.read_csv(fcst_q_98)
fcst_q_9 = pd.read_csv(fcst_q_9)
fcst_q_92 = pd.read_csv(fcst_q_92)
fcst_q_94 = pd.read_csv(fcst_q_94)
fcst_q_96 = pd.read_csv(fcst_q_96)
print(fcst_q_98.shape[0],fcst_q_96.shape[0],fcst_q_94.shape[0],fcst_q_92.shape[0],fcst_q_9.shape[0])





fcst.rename(columns={"predict": "fcst"},inplace=True)
fcst_q_98.rename(columns={"predict": "quantile_fcst"},inplace=True)
fcst_q_98['quantile_ratio'] = 0.98
fcst_q_96.rename(columns={"predict": "quantile_fcst"},inplace=True)
fcst_q_96['quantile_ratio'] = 0.96
fcst_q_94.rename(columns={"predict": "quantile_fcst"},inplace=True)
fcst_q_94['quantile_ratio'] = 0.94
fcst_q_92.rename(columns={"predict": "quantile_fcst"},inplace=True)
fcst_q_92['quantile_ratio'] = 0.92
fcst_q_9.rename(columns={"predict": "quantile_fcst"},inplace=True)
fcst_q_9['quantile_ratio'] = 0.9
fcst_98 = pd.merge(fcst,fcst_q_98,how="inner", on=["sku_id", "paytime_day", "target_qty_m",'coverage'])
fcst_96 = pd.merge(fcst,fcst_q_96,how="inner", on=["sku_id", "paytime_day", "target_qty_m",'coverage'])
fcst_94 = pd.merge(fcst,fcst_q_94,how="inner", on=["sku_id", "paytime_day", "target_qty_m",'coverage'])
fcst_92 = pd.merge(fcst,fcst_q_92,how="inner", on=["sku_id", "paytime_day", "target_qty_m",'coverage'])
fcst_9 = pd.merge(fcst,fcst_q_9,how="inner", on=["sku_id", "paytime_day", "target_qty_m",'coverage'])
fcst = pd.concat([fcst_98, fcst_96,fcst_94,fcst_92,fcst_9],axis=0,ignore_index=False)
print(fcst.shape,fcst_q_98.shape,fcst_q_96.shape,fcst_q_94.shape,fcst_q_92.shape,fcst_q_9.shape)
fcst["date"] = pd.to_datetime(fcst["paytime_day"])

product_info = pd.read_csv("product_info_20190618.csv")
product_info = product_info[["sku_id", "family_label_zh"]].drop_duplicates()
fcst = pd.merge(fcst, product_info, how="inner", on="sku_id")

reg_list = pd.read_csv("df_replenish_params_20190626.csv")
reg_list = reg_list[["sku_id", "class"]].drop_duplicates()
fcst = pd.merge(fcst, reg_list, how="inner", on="sku_id")

boolean = (fcst["class"] == "reg") & \
          (fcst["family_label_zh"].isin(["健身椭圆机", "女士马拉松跑服装"])) &  \
          (fcst["date"] >= "2019-02-05") & \
          (fcst["date"] <= "2019-03-31")
fcst.loc[boolean, "fcst"] = fcst[boolean]["fcst"] * times
fcst.loc[boolean, "quantile_fcst"] = fcst[boolean]["quantile_fcst"] * times

fcst = fcst.rename(columns={"paytime_day": "datetime", "target_qty": "qty"})
fcst.to_csv(save_path,index=False)
# Save
# fcst.to_csv(save_path, index=False)

# # Reference
# fcst <- fread("promo_label/result_m_0.98_20190626.csv")
# fcst[, date := as.Date(datetime)]
# fcst <- merge(fcst, rep[, .(sku_id, class)], by = "sku_id")
# fcst <- merge(fcst, sku[, .(sku_id, family_label_zh)], by = "sku_id")
# fcst[class == "reg" & 
#      family_label_zh %in% c("健身椭圆机", "女士马拉松跑服装") & 
#      date %between% c("2019-02-05", "2019-03-31"), 
#      fcst := fcst * 1.5]
# fcst[class == "reg" & 
#      family_label_zh %in% c("健身椭圆机", "女士马拉松跑服装") & 
#      date %between% c("2019-02-05", "2019-03-31"), 
#      quantile_fcst := quantile_fcst * 1.5]
# write.csv(fcst, "promo_label/df_fcst.csv", row.names = FALSE, fileEncoding = "UTF-8")

In [85]:
fcst_q_98.head(1)

,sku_id,paytime_day,target_qty_m,coverage,quantile_fcst,quantile_ratio
0,2698460,2019-03-26,1.0,1,3.069455,0.98


In [24]:
order = pd.read_csv('order_20190618.csv')
order = order[order['stock_name_cn']=='酷武花桥']
order.head(5)

,order_bn,sku_id,tmall_id,product_tmall_name,logi_name,delivery_id,ship_time,paytime,ship_province,ship_city,channel,qty,original_price,original_total_price,sales_amount,stock_name_cn
0,1491694757850737,534154,4.383156e+10,迪卡侬 新款 跑步鞋袋 便携轻便大容量 双肩收纳袋 KALENJI,圆通速递,1512241104027,2015-12-24 14:37:38,2015-12-22 12:48:46,上海,上海市,Decathlon天猫旗舰店,1,19.9,19.9,19.90,酷武花桥
1,1503019550171731,534154,4.383156e+10,迪卡侬 新款 跑步鞋袋 便携轻便大容量 双肩收纳袋 KALENJI,圆通速递,1512261103689,2015-12-26 16:47:40,2015-12-26 14:55:30,山东,青岛市,Decathlon天猫旗舰店,1,19.9,19.9,19.90,酷武花桥
2,1524788171634282,534154,4.383156e+10,迪卡侬 新款 跑步鞋袋 便携轻便大容量 双肩收纳袋 KALENJI,圆通速递,1601041105582,2016-01-05 10:47:43,2016-01-03 10:46:03,北京,北京市,Decathlon天猫旗舰店,1,19.9,19.9,19.49,酷武花桥
3,1327109182510690,534154,4.383156e+10,迪卡侬 新款 跑步鞋袋 便携轻便大容量 双肩收纳袋 KALENJI,圆通速递,1601101104006,2016-01-10 17:07:41,2016-01-06 18:39:16,上海,上海市,Decathlon天猫旗舰店,1,19.9,19.9,19.37,酷武花桥
10,1572053123839952,534154,4.383156e+10,迪卡侬 新款 跑步鞋袋 便携轻便大容量 双肩收纳袋 KALENJI,圆通速递,1601191109300,2016-01-19 20:38:01,2016-01-18 20:28:55,安徽,淮南市,Decathlon天猫旗舰店,1,19.9,19.9,19.90,酷武花桥


In [25]:
larry = pd.read_excel('larry.xlsx')
larry.head(5)

,order_bn,larry_list,filter_list,anew_list,anew_list_m3,anew_list_m2
0,201809211410009965,1.0,NaN,NaN,NaN,NaN
1,201809251610001968,1.0,NaN,NaN,NaN,NaN
2,201811281610009502,1.0,NaN,NaN,NaN,NaN
3,201812141710009917,1.0,NaN,NaN,NaN,NaN
4,201812181310008665,1.0,NaN,NaN,NaN,NaN


In [36]:
data = order.merge(larry,how='left',on='order_bn')
#data['date'] = data['paytime'].map(lambda x:x.strftime('%Y-%m-%d'))
data['date'] = data['paytime'].map(lambda x:x[:10])
#data['date'] = pd.to_datetime(data['date'])
data.head(5)

,order_bn,sku_id,tmall_id,product_tmall_name,logi_name,delivery_id,ship_time,paytime,ship_province,ship_city,...,original_price,original_total_price,sales_amount,stock_name_cn,larry_list,filter_list,anew_list,anew_list_m3,anew_list_m2,date
0,1491694757850737,534154,4.383156e+10,迪卡侬 新款 跑步鞋袋 便携轻便大容量 双肩收纳袋 KALENJI,圆通速递,1512241104027,2015-12-24 14:37:38,2015-12-22 12:48:46,上海,上海市,...,19.9,19.9,19.90,酷武花桥,NaN,NaN,NaN,NaN,NaN,2015-12-22
1,1503019550171731,534154,4.383156e+10,迪卡侬 新款 跑步鞋袋 便携轻便大容量 双肩收纳袋 KALENJI,圆通速递,1512261103689,2015-12-26 16:47:40,2015-12-26 14:55:30,山东,青岛市,...,19.9,19.9,19.90,酷武花桥,NaN,NaN,NaN,NaN,NaN,2015-12-26
2,1524788171634282,534154,4.383156e+10,迪卡侬 新款 跑步鞋袋 便携轻便大容量 双肩收纳袋 KALENJI,圆通速递,1601041105582,2016-01-05 10:47:43,2016-01-03 10:46:03,北京,北京市,...,19.9,19.9,19.49,酷武花桥,NaN,NaN,NaN,NaN,NaN,2016-01-03
3,1327109182510690,534154,4.383156e+10,迪卡侬 新款 跑步鞋袋 便携轻便大容量 双肩收纳袋 KALENJI,圆通速递,1601101104006,2016-01-10 17:07:41,2016-01-06 18:39:16,上海,上海市,...,19.9,19.9,19.37,酷武花桥,NaN,NaN,NaN,NaN,NaN,2016-01-06
4,1572053123839952,534154,4.383156e+10,迪卡侬 新款 跑步鞋袋 便携轻便大容量 双肩收纳袋 KALENJI,圆通速递,1601191109300,2016-01-19 20:38:01,2016-01-18 20:28:55,安徽,淮南市,...,19.9,19.9,19.90,酷武花桥,NaN,NaN,NaN,NaN,NaN,2016-01-18


In [37]:
data_larry = data[(data['larry_list']==1)&(data['filter_list']!=1)]
data_larry.head(5)

,order_bn,sku_id,tmall_id,product_tmall_name,logi_name,delivery_id,ship_time,paytime,ship_province,ship_city,...,original_price,original_total_price,sales_amount,stock_name_cn,larry_list,filter_list,anew_list,anew_list_m3,anew_list_m2,date
5499,cn445739927,2172274,NaN,有氧健身前置飞轮智能多功能训练器械室内太空漫步家用磁控静音心率可测高端机成人椭圆机E Ene...,德邦物流,1809051111204,2018-09-05 17:16:29,2018-09-05 11:39:20,上海,上海市,...,4599.9,4599.9,4599.9,酷武花桥,1.0,NaN,NaN,NaN,NaN,2018-09-05
5657,201809211410009965,2607008,0.0,FEL 120 CN,顺丰速运,1809231105993,2018-09-23 12:13:03,2018-09-21 14:57:19,上海,上海市,...,1599.9,1599.9,1599.9,酷武花桥,1.0,NaN,NaN,NaN,NaN,2018-09-21
5675,201809251610001968,2172274,0.0,ELLIPTICAL E ENERGY CN,大件物流,1809261107979,2018-09-26 18:03:11,2018-09-25 16:07:45,上海,上海市,...,4599.9,4599.9,4599.9,酷武花桥,1.0,NaN,NaN,NaN,NaN,2018-09-25
6373,cn520434775,2172274,NaN,有氧健身前置飞轮智能多功能训练器械室内太空漫步家用磁控静音心率可测高端机成人椭圆机E Ene...,德邦物流,1811061114587,2018-11-06 20:51:56,2018-11-06 18:01:58,浙江,舟山市,...,4599.9,4599.9,4599.9,酷武花桥,1.0,NaN,NaN,NaN,NaN,2018-11-06
7722,201812141710009917,2172274,0.0,ELLIPTICAL E ENERGY CN,德邦物流,1812161116225,2018-12-16 18:47:58,2018-12-14 17:50:08,上海,上海市,...,4599.9,4599.9,4599.9,酷武花桥,1.0,NaN,NaN,NaN,NaN,2018-12-14


In [34]:
sku_data.head(1)

,date,qty,is_group
0,2019-01-30,1.0,NaN


In [38]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

with PdfPages('case_3.pdf') as pdf:
    for sku in data_larry['sku_id'].unique():
        plt.figure(figsize=(15, 5))
        sku_data = data[data['sku_id']==sku]
        min_date = np.min(sku_data['date'])
        max_date = np.max(sku_data['date'])
        mark = sku_data[(sku_data['larry_list']==1)&(sku_data['filter_list']!=1)]['date'].unique().tolist()
        #print(mark)
        #print(min_date,max_date)
        date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=min_date, end=max_date))]
        #print(date_list)
        sku_data = sku_data.groupby('date',as_index=False).qty.sum()
        sku_data.set_index('date',inplace=True)
        #print(sku_data.iloc[:5])
        sku_data = sku_data.reindex(date_list).fillna(0).reset_index()
        sku_data['date'] = pd.to_datetime(sku_data['date'])
        #del sku_data['index']
        #print(sku_data.iloc[:5])
        sku_larry = data_larry[data_larry['sku_id']==sku]
        sku_data.sort_values(by='date',inplace=True)

        sku_data.loc[sku_data["date"].isin(mark), "is_group"] = 1

        plt.plot(sku_data['date'],sku_data['qty'])
        plt.scatter(sku_data[sku_data["is_group"] == 1]["date"].values, 
                    sku_data[sku_data["is_group"] == 1]["qty"].values,
                    marker="o", c="red", s=20, label="group")
        plt.legend()
        #for i in mark:
        #   plt.plot(i,sku_data[sku_data['date']==i]['qty'],'r')
        #plt.plot(sku_larry['paytime'])
        plt.title(sku)
        pdf.savefig()
        plt.close()
    print('complete')

complete


In [31]:
sku_data[sku_data["is_group"] == 1]["date"]
# sku_data[sku_data["is_group"] == 1]["qty"]

526   2018-09-05
546   2018-09-25
588   2018-11-06
626   2018-12-14
631   2018-12-19
647   2019-01-04
658   2019-01-15
737   2019-04-04
761   2019-04-28
764   2019-05-01
769   2019-05-06
777   2019-05-14
Name: date, dtype: datetime64[ns]

In [94]:
kpi = pd.read_csv('kpi_summary.csv')
kpi.head(7)

,Unnamed: 0,sku_count,turn_day,weight_turn_day,spot_rate,weight_spot_rate,satisfied_rate,weight_satisfied_rate,order_freq,mean_inventory,max_available_inventory,mean_instock
0,1,75,69.58,12.854816,99.97%,99.8%,99.78%,97.99%,0.17,9.93,15.88,2.49
1,2,75,51.91,7.202225,99.87%,99.31%,99.29%,96.52%,0.25,5.56,9.27,1.58
2,3,75,51.89,7.188138,99.87%,99.18%,99.34%,96.54%,0.25,5.55,9.27,1.53
3,4,75,51.89,7.189427,99.88%,99.38%,99.33%,96.43%,0.25,5.55,9.27,1.53
4,5,75,51.89,7.189524,99.88%,99.38%,99.33%,96.42%,0.25,5.55,9.27,1.54
5,6,75,51.89,7.190402,99.88%,99.38%,99.31%,96.34%,0.25,5.55,9.27,1.55
6,7,75,45.29,6.345161,99.26%,98.12%,98.46%,95.5%,0.25,4.90,8.12,1.51


In [121]:
kpi_class = pd.read_csv('kpi_summary_by_class.csv')
kpi_class_real = pd.read_csv('kpi_summary_by_class_real.csv')
kpi_class['No.'] = kpi_class.index//7+1
kpi_class['weight_spot_rate'] = kpi_class['weight_spot_rate'].map(lambda x:x[:-1]).astype(float)/100
kpi_class['weight_satisfied_rate'] = kpi_class['weight_satisfied_rate'].map(lambda x:x[:-1]).astype(float)/100
kpi_class_real['weight_spot_rate'] = kpi_class_real['weight_spot_rate'].map(lambda x:x[:-1]).astype(float)/100
kpi_class_real['weight_satisfied_rate'] = kpi_class_real['weight_satisfied_rate'].map(lambda x:x[:-1]).astype(float)/100
kpi_class

,Unnamed: 0,family_id,class,sku_count,turn_day,weight_turn_day,spot_rate,weight_spot_rate,satisfied_rate,weight_satisfied_rate,order_freq,mean_inventory,max_available_inventory,mean_instock,No.
0,1,1,lgt,31,56.84,36.11,100%,1.0000,100%,1.0000,0.09,6.72,9.97,1.90,1
1,2,1,reg,18,32.39,11.51,100%,1.0000,100%,1.0000,0.33,15.71,25.56,3.52,1
2,3,2,lgt,17,153.88,93.35,100%,1.0000,100%,1.0000,0.02,4.09,4.53,0.83,1
3,4,2,reg,1,108.00,108.00,100%,1.0000,100%,1.0000,0.02,2.23,4.00,1.50,1
4,5,3,reg,5,7.97,5.94,99.76%,0.9951,97.64%,0.9516,0.59,28.36,55.40,7.29,1
5,6,4,lgt,1,64.75,64.75,100%,1.0000,100%,1.0000,0.03,6.88,9.00,4.00,1
6,7,4,reg,2,22.26,11.53,99.56%,0.9994,97.73%,0.9969,0.26,16.43,27.50,4.20,1
7,8,1,lgt,31,34.16,21.30,99.94%,0.9991,99.78%,0.9969,0.13,3.97,5.77,1.25,2
8,9,1,reg,18,17.47,5.99,100%,1.0000,99.47%,0.9906,0.47,8.17,15.39,2.18,2
9,10,2,lgt,17,138.93,74.98,100%,1.0000,100%,1.0000,0.02,3.28,3.53,0.63,2


In [131]:
reg2 = kpi_class.loc[(kpi_class['class']=='reg')&(kpi_class['No.']==2)]
lgt2 = kpi_class.loc[(kpi_class['class']=='lgt')&(kpi_class['No.']==2)]
reg7 = kpi_class.loc[(kpi_class['class']=='reg')&(kpi_class['No.']==7)]
lgt7 =kpi_class.loc[(kpi_class['class']=='lgt')&(kpi_class['No.']==7)]
regr = kpi_class_real.loc[kpi_class_real['class']=='reg']
lgtr = kpi_class_real.loc[kpi_class_real['class']=='lgt']
'''
reg_weight_turn_day = np.multiply(reg['weight_turn_day'],reg['sku_count'])/np.sum(reg['sku_count'])
lgt_weight_turn_day = np.multiply(lgt['weight_turn_day'],lgt['sku_count'])/np.sum(lgt['sku_count'])
reg_weight_spot_rate = np.multiply(reg['weight_spot_rate'],reg['sku_count'])/np.sum(reg['sku_count'])
lgt_weight_spot_rate = np.multiply(lgt['weight_spot_rate'],lgt['sku_count'])/np.sum(lgt['sku_count'])
reg_weight_sat_rate = np.multiply(reg['weight_satisfied_rate'],reg['sku_count'])/np.sum(reg['sku_count'])
lgt_weight_sat_rate = np.multiply(lgt['weight_satisfied_rate'],lgt['sku_count'])/np.sum(lgt['sku_count'])
reg_mean_inventory = 
'''
reg_weight_turn_day = [np.dot(reg2['sku_count'],reg2['weight_turn_day'])/np.sum(reg2['sku_count']),
                       np.dot(reg7['sku_count'],reg7['weight_turn_day'])/np.sum(reg7['sku_count']),
                       np.dot(regr['sku_count'],regr['weight_turn_day'])/np.sum(regr['sku_count'])] 
lgt_weight_turn_day = [np.dot(lgt2['sku_count'],lgt2['weight_turn_day'])/np.sum(lgt2['sku_count']),
                       np.dot(lgt7['sku_count'],lgt7['weight_turn_day'])/np.sum(lgt7['sku_count']),
                       np.dot(lgtr['sku_count'],lgtr['weight_turn_day'])/np.sum(lgtr['sku_count'])] 

reg_weight_spot_rate = [np.dot(reg2['sku_count'],reg2['weight_spot_rate'])/np.sum(reg2['sku_count']),
                       np.dot(reg7['sku_count'],reg7['weight_spot_rate'])/np.sum(reg7['sku_count']),
                       np.dot(regr['sku_count'],regr['weight_spot_rate'])/np.sum(regr['sku_count'])]
lgt_weight_spot_rate = [np.dot(lgt2['sku_count'],lgt2['weight_spot_rate'])/np.sum(lgt2['sku_count']),
                       np.dot(lgt7['sku_count'],lgt7['weight_spot_rate'])/np.sum(lgt7['sku_count']),
                       np.dot(lgtr['sku_count'],lgtr['weight_spot_rate'])/np.sum(lgtr['sku_count'])] 

reg_weight_satisfied_rate = [np.dot(reg2['sku_count'],reg2['weight_satisfied_rate'])/np.sum(reg2['sku_count']),
                       np.dot(reg7['sku_count'],reg7['weight_satisfied_rate'])/np.sum(reg7['sku_count']),
                       np.dot(regr['sku_count'],regr['weight_satisfied_rate'])/np.sum(regr['sku_count'])]
lgt_weight_satisfied_rate = [np.dot(lgt2['sku_count'],lgt2['weight_satisfied_rate'])/np.sum(lgt2['sku_count']),
                       np.dot(lgt7['sku_count'],lgt7['weight_satisfied_rate'])/np.sum(lgt7['sku_count']),
                       np.dot(lgtr['sku_count'],lgtr['weight_satisfied_rate'])/np.sum(lgtr['sku_count'])]

reg_mean_inventory = [np.dot(reg2['sku_count'],reg2['mean_inventory'])/np.sum(reg2['sku_count']),
                       np.dot(reg7['sku_count'],reg7['mean_inventory'])/np.sum(reg7['sku_count']),
                       np.dot(regr['sku_count'],regr['mean_inventory'])/np.sum(regr['sku_count'])]
lgt_mean_inventory = [np.dot(lgt2['sku_count'],lgt2['mean_inventory'])/np.sum(lgt2['sku_count']),
                       np.dot(lgt7['sku_count'],lgt7['mean_inventory'])/np.sum(lgt7['sku_count']),
                       np.dot(lgtr['sku_count'],lgtr['mean_inventory'])/np.sum(lgtr['sku_count'])]

reg_max_inventory = [np.dot(reg2['sku_count'],reg2['max_available_inventory'])/np.sum(reg2['sku_count']),
                       np.dot(reg7['sku_count'],reg7['max_available_inventory'])/np.sum(reg7['sku_count']),
                       np.dot(regr['sku_count'],regr['max_available_inventory'])/np.sum(regr['sku_count'])]
lgt_max_inventory = [np.dot(lgt2['sku_count'],lgt2['max_available_inventory'])/np.sum(lgt2['sku_count']),
                       np.dot(lgt7['sku_count'],lgt7['max_available_inventory'])/np.sum(lgt7['sku_count']),
                       np.dot(lgtr['sku_count'],lgtr['max_available_inventory'])/np.sum(lgtr['sku_count'])]

print(reg_weight_turn_day)
print(lgt_weight_turn_day)
print(reg_weight_spot_rate)
print(lgt_weight_spot_rate)
print(reg_weight_satisfied_rate)
print(lgt_weight_satisfied_rate)
print(reg_mean_inventory)
print(lgt_mean_inventory)
print(reg_max_inventory)
print(lgt_max_inventory)





[7.224615384615385, 7.127692307692309, 7.7042307692307705]
[40.701632653061225, 31.688163265306127, 31.947959183673472]
[0.996423076923077, 0.9807076923076923, 0.9933269230769229]
[0.9994306122448979, 0.9846122448979591, 0.9873714285714286]
[0.9786153846153847, 0.9674, 1.0]
[0.998038775510204, 0.9764020408163265, 1.0]
[8.924615384615386, 8.8, 11.925769230769232]
[3.7783673469387757, 2.8355102040816327, 3.0218367346938773]
[17.23153846153846, 17.23153846153846, 33.00076923076923]
[5.038367346938775, 3.282244897959184, 5.472244897959183]


In [135]:
kpi_bycodee = pd.read_csv('kpi_summary_by_codee.csv')
kpi_bycodee['No.'] = kpi_bycodee.index//8+1
kpi_bycodee_real = pd.read_csv('kpi_summary_by_codee_real.csv')
kpi_bycodee['weight_spot_rate'] = kpi_bycodee['weight_spot_rate'].map(lambda x:x[:-1]).astype(float)/100
kpi_bycodee['weight_satisfied_rate'] = kpi_bycodee['weight_satisfied_rate'].map(lambda x:x[:-1]).astype(float)/100
kpi_bycodee_real['weight_spot_rate'] = kpi_bycodee_real['weight_spot_rate'].map(lambda x:x[:-1]).astype(float)/100
kpi_bycodee_real['weight_satisfied_rate'] = kpi_bycodee_real['weight_satisfied_rate'].map(lambda x:x[:-1]).astype(float)/100
kpi_bycodee

,Unnamed: 0,family_id,code_e,sku_count,turn_day,weight_turn_day,spot_rate,weight_spot_rate,satisfied_rate,weight_satisfied_rate,order_freq,mean_inventory,max_available_inventory,mean_instock,No.
0,1,1,E,9,36.58,8.46,100%,1.0000,100%,1.0000,0.30,14.28,26.11,3.62,1
1,2,1,N,40,50.40,23.95,100%,1.0000,100%,1.0000,0.15,9.07,13.35,2.24,1
2,3,2,E,2,80.75,80.84,100%,1.0000,100%,1.0000,0.04,4.27,5.00,0.67,1
3,4,2,N,16,160.15,95.81,100%,1.0000,100%,1.0000,0.02,3.95,4.44,0.89,1
4,5,3,E,4,6.19,5.58,99.7%,0.9949,97.05%,0.9497,0.64,32.05,62.25,8.49,1
5,6,3,N,1,15.08,15.08,100%,1.0000,100%,1.0000,0.36,13.61,28.00,2.51,1
6,7,4,E,1,9.83,9.83,100%,1.0000,100%,1.0000,0.42,26.11,45.00,6.57,1
7,8,4,N,2,49.72,45.29,99.56%,0.9943,97.73%,0.9706,0.07,6.81,9.50,2.92,1
8,9,1,E,9,20.78,4.49,100%,1.0000,98.79%,0.9857,0.45,7.58,14.11,2.39,2
9,10,1,N,40,29.66,13.32,99.95%,0.9997,99.87%,0.9978,0.21,5.05,8.22,1.41,2


In [136]:
e2 = kpi_bycodee.loc[(kpi_bycodee['code_e']=='E')&(kpi_bycodee['No.']==2)]
n2 = kpi_bycodee.loc[(kpi_bycodee['code_e']=='N')&(kpi_bycodee['No.']==2)]
e7 = kpi_bycodee.loc[(kpi_bycodee['code_e']=='E')&(kpi_bycodee['No.']==7)]
n7 =kpi_bycodee.loc[(kpi_bycodee['code_e']=='N')&(kpi_bycodee['No.']==7)]
er = kpi_bycodee_real.loc[kpi_bycodee_real['code_e']=='E']
nr = kpi_bycodee_real.loc[kpi_bycodee_real['code_e']=='N']

e_weight_turn_day = [np.dot(e2['sku_count'],e2['weight_turn_day'])/np.sum(e2['sku_count']),
                       np.dot(e7['sku_count'],e7['weight_turn_day'])/np.sum(e7['sku_count']),
                       np.dot(er['sku_count'],er['weight_turn_day'])/np.sum(er['sku_count'])] 
n_weight_turn_day = [np.dot(n2['sku_count'],n2['weight_turn_day'])/np.sum(n2['sku_count']),
                       np.dot(n7['sku_count'],n7['weight_turn_day'])/np.sum(n7['sku_count']),
                       np.dot(nr['sku_count'],nr['weight_turn_day'])/np.sum(nr['sku_count'])] 

e_weight_spot_rate = [np.dot(e2['sku_count'],e2['weight_spot_rate'])/np.sum(e2['sku_count']),
                       np.dot(e7['sku_count'],e7['weight_spot_rate'])/np.sum(e7['sku_count']),
                       np.dot(er['sku_count'],er['weight_spot_rate'])/np.sum(er['sku_count'])]
n_weight_spot_rate = [np.dot(n2['sku_count'],n2['weight_spot_rate'])/np.sum(n2['sku_count']),
                       np.dot(n7['sku_count'],n7['weight_spot_rate'])/np.sum(n7['sku_count']),
                       np.dot(nr['sku_count'],nr['weight_spot_rate'])/np.sum(nr['sku_count'])] 

e_weight_satisfied_rate = [np.dot(e2['sku_count'],e2['weight_satisfied_rate'])/np.sum(e2['sku_count']),
                       np.dot(e7['sku_count'],e7['weight_satisfied_rate'])/np.sum(e7['sku_count']),
                       np.dot(er['sku_count'],er['weight_satisfied_rate'])/np.sum(er['sku_count'])]
n_weight_satisfied_rate = [np.dot(n2['sku_count'],n2['weight_satisfied_rate'])/np.sum(n2['sku_count']),
                       np.dot(n7['sku_count'],n7['weight_satisfied_rate'])/np.sum(n7['sku_count']),
                       np.dot(nr['sku_count'],nr['weight_satisfied_rate'])/np.sum(nr['sku_count'])]

e_mean_inventory = [np.dot(e2['sku_count'],e2['mean_inventory'])/np.sum(e2['sku_count']),
                       np.dot(e7['sku_count'],e7['mean_inventory'])/np.sum(e7['sku_count']),
                       np.dot(er['sku_count'],er['mean_inventory'])/np.sum(er['sku_count'])]
n_mean_inventory = [np.dot(n2['sku_count'],n2['mean_inventory'])/np.sum(n2['sku_count']),
                       np.dot(n7['sku_count'],n7['mean_inventory'])/np.sum(n7['sku_count']),
                       np.dot(nr['sku_count'],nr['mean_inventory'])/np.sum(nr['sku_count'])]

e_max_inventory = [np.dot(e2['sku_count'],e2['max_available_inventory'])/np.sum(e2['sku_count']),
                       np.dot(e7['sku_count'],e7['max_available_inventory'])/np.sum(e7['sku_count']),
                       np.dot(er['sku_count'],er['max_available_inventory'])/np.sum(er['sku_count'])]
n_max_inventory = [np.dot(n2['sku_count'],n2['max_available_inventory'])/np.sum(n2['sku_count']),
                       np.dot(n7['sku_count'],n7['max_available_inventory'])/np.sum(n7['sku_count']),
                       np.dot(nr['sku_count'],nr['max_available_inventory'])/np.sum(nr['sku_count'])]

print(e_weight_turn_day)
print(n_weight_turn_day)
print(e_weight_spot_rate)
print(n_weight_spot_rate)
print(e_weight_satisfied_rate)
print(n_weight_satisfied_rate)
print(e_mean_inventory)
print(n_mean_inventory)
print(e_max_inventory)
print(n_max_inventory)




[10.8975, 7.82, 10.30375]
[31.150508474576274, 26.2264406779661, 24.915254237288135]
[0.99575, 0.9747687500000001, 0.9943187499999999]
[0.999522033898305, 0.9936372881355932, 0.9906847457627118]
[0.97313125, 0.95749375, 1.0]
[0.9945220338983051, 0.986928813559322, 1.0]
[9.236875000000001, 8.611250000000002, 15.01375]
[4.569152542372881, 3.8932203389830513, 3.6899999999999995]
[17.561875, 16.125625, 41.9975]
[7.014237288135594, 5.9491525423728815, 7.698305084745763]
